In [283]:
import os
import pandas as pd
import xarray as xr
import glob
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import sqlite3

# Metadata database

In [392]:
#%% Get station metadata

DB_NAME = "/kfs2/projects/sfcwinds/data/weather_data_expanded.db" # if things fail in this script, start over with the backup.

def list_tables():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    conn.close()
    return tables

tables = list_tables()
if tables:
    print("Tables in the database:")
    for table in tables:
        print(f" - {table[0]}")
else:
    print("No tables found in the database.")


def load_table_to_df(table_name):
    conn = sqlite3.connect(DB_NAME)
    
    # Load the table into a DataFrame
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    
    conn.close()
    return df

# Load the stations table into a DataFrame
stations_df = load_table_to_df("stations") #  "stations" or "data_references", data_references includes only Ameriflux stations.

print("Stations:")
print(stations_df)



Tables in the database:
 - data_references
 - stations
Stations:
     station_id                                  station_name        lat  \
0           EAS                                          None  37.871000   
1           HES                                          None  38.133300   
2           MOB                                          None  37.773344   
3           POW                                          None  39.800000   
4           SCO                                          None  38.534669   
...         ...                                           ...        ...   
3281     US-ASH  USSL San Joaquin Valley Almond High Salinity  36.169700   
3282     US-An2                 Anaktuvuk River Moderate Burn  68.950000   
3283     US-Ro5                           Rosemount I18_South  44.691000   
3284     US-MN3    Morris: Corn-Soybean, Conventional Tillage  45.609100   
3285     US-CF3                           CAF-LTAR Boyd North  46.755100   

             lon  heig

In [336]:
# List of networks in the database
np.unique(np.array([x for x in stations_df.source_network.values if x is not None]))

array(['AGRIMET', 'AVALANCHE', 'AZMET', 'AmeriFlux', 'BTAVAL', 'CCU',
       'CLNP', 'CRN', 'CoAgMet', 'DEOS', 'DRI', 'FGNet', 'HADS', 'HMMN',
       'IEM', 'IGS', 'IUTAH', 'LSU-AGNET', 'MAWN', 'MISC', 'MOCOMAGNET',
       'MT-MESO', 'NICEnet', 'NJNET', 'NMCC', 'PEMN', 'PURDUE', 'RAWS',
       'SCAN', 'SDGE', 'TWDB', 'UCC-AGNET', 'UGA', 'USARRAY', 'USCRN',
       'UTAH CLIMATE CENTER', 'UTAH DOT', 'UUNET'], dtype='<U19')

In [242]:
# First, fix the state names and binary heights in the metadata  - ok if done once (saved back to database)

# Connect to the SQLite database
db_path = "/kfs2/projects/sfcwinds/data/weather_data_expanded.db"
conn = sqlite3.connect(db_path)

print(f"Connected to database: {db_path}")

# Load the stations table into a DataFrame
try:
    stations_df = pd.read_sql_query("SELECT * FROM stations", conn)
    print("Successfully loaded 'stations' table.")
except Exception as e:
    print(f"Error loading 'stations' table: {e}")
    conn.close()
    exit()

# --- State Standardization ---
print("\nStandardizing 'state' column...")

# Define the standardization function
def standardize_state(state_code):
    if isinstance(state_code, str): # Ensure it's a string before operations
        if state_code == 'A1':
            return 'AL'
        return state_code.upper()
    return state_code # Return as is if not a string (e.g., NaN)

# Apply the standardization function to the 'state' column
stations_df['state'] = stations_df['state'].apply(standardize_state)

print("State standardization complete.")

# Display a sample of the updated states to verify
print("\nSample of updated 'state' column (first 10 entries):")
print(stations_df['state'].value_counts().head(10)) # Show top 10 most frequent states after update


# --- Convert 'height' column (some data have height as binary data) ---
import struct # Import the struct module for unpacking binary data
print("\nConverting 'height' column to numeric values...")

def convert_height(value):
    if value is None:
        return None  # Keep None values as they are (or pd.NA)
    elif isinstance(value, bytes):
        # Assuming 8-byte little-endian signed integer for binary data
        try:
            # Unpack as a long long (q for signed long long, Q for unsigned)
            # Use '<' for little-endian byte order
            return float(struct.unpack('<q', value)[0])
        except struct.error:
            # Handle cases where binary data might not be exactly 8 bytes or malformed
            print(f"Warning: Could not unpack malformed binary height value: {value}")
            return None # Or return pd.NA for clarity on failure
        except Exception as e:
            print(f"An unexpected error occurred converting binary height {value}: {e}")
            return None
    else:
        # Convert string representations of numbers to float
        try:
            return float(value)
        except (ValueError, TypeError):
            # Handle cases where string might not be a valid number
            print(f"Warning: Could not convert non-numeric string height value: '{value}'")
            return None # Or return pd.NA

# Apply the conversion function to the 'height' column
stations_df['height'] = stations_df['height'].apply(convert_height)

print("Height conversion complete.")

# Display a sample of the updated heights
print("\nSample of updated 'height' column:")
print(stations_df[['station_name', 'height']])

print("\nValue counts for the 'height' column after conversion (top 10):")
print(stations_df['height'].value_counts(dropna=False).head(10))

# --- Saving Changes Back to Database ---
print("\nSaving updated 'stations' table back to the database...")
try:
    # Save the entire modified DataFrame back to the database, replacing the old table.
    # This will update the 'state' column while keeping other columns and rows intact.
    stations_df.to_sql(name='stations', con=conn, if_exists='replace', index=False)
    print("\n---")
    print("### Database Update Complete")
    print("---")
    print("The 'state' column in the 'stations' table has been standardized, heights converted to readable numbers.")
except Exception as e:
    print(f"\n---")
    print(f"### Database Update Failed: {e}")
    print(f"---")

# Close the database connection
conn.close()


Connected to database: /kfs2/projects/sfcwinds/data/weather_data_expanded.db
Successfully loaded 'stations' table.

Standardizing 'state' column...
State standardization complete.

Sample of updated 'state' column (first 10 entries):
state
CA    527
UT    226
CO    221
AK    196
MT    196
OR    163
NM    147
GA    123
ID    119
AZ    113
Name: count, dtype: int64

Converting 'height' column to numeric values...
Height conversion complete.

Sample of updated 'height' column:
                station_name  height
0              East Sedgwick     2.0
1                    Hesston     2.0
2             Mobile_Station     2.0
3                  Powhattan     2.0
4                      Scott     2.0
...                      ...     ...
3250                   Hondo     3.0
3251               Canoncito     3.0
3252           Cubero 4.2NNE     3.0
3253          Acoma Sky City     3.0
3254  Iyanbito Chapter House     3.0

[3255 rows x 2 columns]

Value counts for the 'height' column after conversi

In [365]:
print(*sorted(stations_df['state'].dropna().unique()))   # the * prints all in one line


AK AL AR AS AZ CA CO DE FL GA GU HI IA ID IL IN KS KY LA MA MD ME MI MN MO MS MT NC ND NE NH NJ NM NV NY OH OK ON OR PA PR RI SA SC SD TN TX UT VA VI VT WA WI WV WY


In [275]:
# Height in USCRN is 1.5m, not 2m

# Connect to the SQLite database
db_path = "/kfs2/projects/sfcwinds/data/weather_data_expanded.db"
conn = sqlite3.connect(db_path)

print(f"Connected to database: {db_path}")

# Load the stations table into a DataFrame
try:
    stations_df = pd.read_sql_query("SELECT * FROM stations", conn)
    print("Successfully loaded 'stations' table.")
except Exception as e:
    print(f"Error loading 'stations' table: {e}")
    conn.close()
    exit()

stations_df.loc[stations_df.source_network == "USCRN", "height"] = 1.5

print(stations_df[stations_df.source_network == "USCRN"].height)


# --- Saving Changes Back to Database ---
print("\nSaving updated 'stations' table back to the database...")
try:
    stations_df.to_sql(name='stations', con=conn, if_exists='replace', index=False)
    print("### Database Update Complete")
except Exception as e:
    print(f"### Database Update Failed: {e}")

# Close the database connection
conn.close()


Connected to database: /kfs2/projects/sfcwinds/data/weather_data_expanded.db
Successfully loaded 'stations' table.
221    1.5
222    1.5
223    1.5
224    1.5
225    1.5
      ... 
378    1.5
379    1.5
380    1.5
381    1.5
382    1.5
Name: height, Length: 162, dtype: float64

Saving updated 'stations' table back to the database...
### Database Update Complete


In [276]:
# Check station heights (had some binary numbers before)
stations_df.height.unique()

array([ 2.  ,  3.  ,  0.  , 10.  ,   nan,  1.5 ,  6.1 ,  4.57,  5.  ,
        7.62,  3.05,  4.  ,  2.5 ,  7.  ,  8.  ,  4.4 ,  2.9 ,  4.6 ,
        5.4 ,  3.8 ,  6.  ,  4.5 ,  3.7 ,  3.5 ,  7.68,  2.95,  2.1 ,
        3.4 ,  3.73,  3.3 ,  5.1 ,  6.53,  3.2 ,  6.4 ,  2.3 ])

In [277]:
plt.hist(stations_df[stations_df.source_network == "CoAgMet"].height)

(array([ 3.,  0., 97., 24.,  0.,  0.,  0.,  0.,  0.,  2.]),
 array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]),
 <BarContainer object of 10 artists>)

In [246]:
# Check station elevations (some are binary, in CoAgMet) - ok if done once (saved back to database)

# Connect to the SQLite database
db_path = "/kfs2/projects/sfcwinds/data/weather_data_expanded.db"
conn = sqlite3.connect(db_path)

print(f"Connected to database: {db_path}")

# Load the stations table into a DataFrame
try:
    stations_df = pd.read_sql_query("SELECT * FROM stations", conn)
    print("Successfully loaded 'stations' table.")
except Exception as e:
    print(f"Error loading 'stations' table: {e}")
    conn.close()
    exit()


def decode_elev(val):
    if isinstance(val, (bytes, bytearray)):
        return float(struct.unpack('<Q', val)[0])  * 0.3048  
    try:
        return float(val)
    except:
        return None

# Apply to the elev column
stations_df["elev_cleaned"] = stations_df["elev"].apply(decode_elev)


# Overwrite 'elev' with the cleaned version
stations_df["elev"] = stations_df["elev_cleaned"]
stations_df = stations_df.drop(columns=["elev_cleaned"])

print ("Changed stations:")
print (stations_df[stations_df.source_network == "CoAgMet"])

print ("MAximum height (m):")
print (print (stations_df[stations_df.source_network == "CoAgMet"].elev.max()))




# --- Saving Changes Back to Database ---
print("\nSaving updated 'stations' table back to the database...")
try:
    # Save the entire modified DataFrame back to the database, replacing the old table.
    stations_df.to_sql(name='stations', con=conn, if_exists='replace', index=False)
    print("\n---")
    print("### Database Update Complete")
    print("---")
except Exception as e:
    print(f"\n---")
    print(f"### Database Update Failed: {e}")
    print(f"---")

# Close the database connection
conn.close()

Connected to database: /kfs2/projects/sfcwinds/data/weather_data_expanded.db
Successfully loaded 'stations' table.
Changed stations:
     station_id              station_name      lat       lon  height  \
5         HOT01  CSU Rogers Mesa Expt Sta  38.7917 -107.7920     2.0   
6         LAR01                    Larand  40.6126 -106.2997     2.0   
7         SLT01                      Silt  39.5667 -107.6934     2.0   
8         LJR01                    LaJara  37.2443 -105.9722     2.0   
9         KRK01                      Kirk  39.6554 -102.6210     2.0   
...         ...                       ...      ...       ...     ...   
126       KLN01                     Kline  37.1261 -108.1465     3.0   
2808      CSU20               CSU - ARDEC  40.6525 -105.0000     3.0   
2809      CSU53   CSU Expt Stn Rocky Ford  38.0385 -103.6950     3.0   
2810      CSU62                     Walsh  37.3830 -102.2940     3.0   
2811      CSU64             Yellow Jacket  37.5289 -108.7240     3.0   

  

In [256]:
# Standardize time format, UTC used everywhere, remove time-zone awareness ( do only once)

# Connect to the SQLite database
db_path = "/kfs2/projects/sfcwinds/data/weather_data_expanded.db"
conn = sqlite3.connect(db_path)

print(f"Connected to database: {db_path}")

# Load the stations table into a DataFrame
try:
    stations_df = pd.read_sql_query("SELECT * FROM stations", conn)
    print("Successfully loaded 'stations' table.")
except Exception as e:
    print(f"Error loading 'stations' table: {e}")
    conn.close()
    exit()


stations_df["begints"] = pd.to_datetime(
    stations_df["begints"], utc=True, format = "mixed", errors="coerce"
).dt.tz_convert(None)

stations_df["endts"] = pd.to_datetime(
    stations_df["endts"], utc=True, format = "mixed", errors="coerce"
).dt.tz_convert(None)



print (stations_df[stations_df.source_network == "CoAgMet"])

# --- Saving Changes Back to Database ---
print("\nSaving updated 'stations' table back to the database...")
try:
    # Save the entire modified DataFrame back to the database, replacing the old table.
    stations_df.to_sql(name='stations', con=conn, if_exists='replace', index=False)
    print("\n---")
    print("### Database Update Complete")
    print("---")
except Exception as e:
    print(f"\n---")
    print(f"### Database Update Failed: {e}")
    print(f"---")

# Close the database connection
conn.close()

Connected to database: /kfs2/projects/sfcwinds/data/weather_data_expanded.db
Successfully loaded 'stations' table.
     station_id              station_name      lat       lon  height  \
5         HOT01  CSU Rogers Mesa Expt Sta  38.7917 -107.7920     2.0   
6         LAR01                    Larand  40.6126 -106.2997     2.0   
7         SLT01                      Silt  39.5667 -107.6934     2.0   
8         LJR01                    LaJara  37.2443 -105.9722     2.0   
9         KRK01                      Kirk  39.6554 -102.6210     2.0   
...         ...                       ...      ...       ...     ...   
126       KLN01                     Kline  37.1261 -108.1465     3.0   
2808      CSU20               CSU - ARDEC  40.6525 -105.0000     3.0   
2809      CSU53   CSU Expt Stn Rocky Ford  38.0385 -103.6950     3.0   
2810      CSU62                     Walsh  37.3830 -102.2940     3.0   
2811      CSU64             Yellow Jacket  37.5289 -108.7240     3.0   

           elev     

In [258]:
# Add the elevation for synoptic data

# Paths
db_path = "/kfs2/projects/sfcwinds/data/weather_data_expanded.db"
csv_path = "/kfs2/projects/sfcwinds/data/general_scripts/US_stations_under_10m.csv"

# Step 1: Read CSV with elevation data
df = pd.read_csv(csv_path)

# Step 2: Rename if needed and convert elevation to float
df = df.rename(columns={
    "STID": "station_id",  
    "elevation_m": "elev"
})


# Step 3: Connect to DB and update
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

updated_count = 0
for _, row in df.iterrows():
    try:
        cursor.execute("""
            UPDATE stations
            SET elev = ?
            WHERE station_id = ?
        """, (row["elev"], row["station_id"]))
        if cursor.rowcount > 0:
            updated_count += 1
    except Exception as e:
        print(f"Failed to update {row['station_id']}: {e}")

conn.commit()
conn.close()

print(f"Finished. Updated elevation for {updated_count} stations.")


Finished. Updated elevation for 2872 stations.


In [378]:
# Fix Ameriflux metadata (replace by the csv that was created seperately)


# Connect to the SQLite database
db_path = "/kfs2/projects/sfcwinds/data/weather_data_expanded.db"
conn = sqlite3.connect(db_path)

print(f"Connected to database: {db_path}")

# Load the stations table into a DataFrame
try:
    stations_df = pd.read_sql_query("SELECT * FROM stations", conn)
    print("Successfully loaded 'stations' table.")
except Exception as e:
    print(f"Error loading 'stations' table: {e}")
    conn.close()
    exit()


# Delete the current Ameriflux metadata
stations_df = stations_df[stations_df["source_network"] != "AmeriFlux"]

# Load the csv with the new metadata
file_path = "/projects/sfcwinds/data/AmeriFlux/ameriflux_metadata.csv"
meta_df = pd.read_csv(file_path)

def parse_begints(val):
    if pd.isna(val):
        return None
    val_str = str(int(val))  # remove '.0'
    if len(val_str) == 8:
        ts = pd.to_datetime(val_str, format="%Y%m%d", errors="coerce")
    elif len(val_str) == 6:
        ts = pd.to_datetime(val_str, format="%y%m%d", errors="coerce")
    else:
        return None

    if pd.isna(ts):
        return None

    return ts.strftime("%Y-%m-%d %H:%M:%S")

meta_df["begints"] = meta_df["begints"].apply(parse_begints)
meta_df["endts"] = meta_df["endts"].apply(parse_begints)

# Add the new metadata to stations_df
stations_df = pd.concat([stations_df, meta_df], ignore_index=True)

# Remove stations outside US
stations_df = stations_df[~stations_df['state'].isin(['SA', 'ON'])]

#print(stations_df[stations_df.source_network == "AmeriFlux"])

# --- Saving Changes Back to Database ---
print("\nSaving updated 'stations' table back to the database...")
try:
    # Save the entire modified DataFrame back to the database, replacing the old table.
    stations_df.to_sql(name='stations', con=conn, if_exists='replace', index=False)
    print("\n---")
    print("### Database Update Complete")
    print("---")
except Exception as e:
    print(f"\n---")
    print(f"### Database Update Failed: {e}")
    print(f"---")

# Close the database connection
conn.close()

Connected to database: /kfs2/projects/sfcwinds/data/weather_data_expanded.db
Successfully loaded 'stations' table.

Saving updated 'stations' table back to the database...

---
### Database Update Complete
---


In [353]:
# NICEnet: height is 3m, state is Nevada

# Connect to the SQLite database
db_path = "/kfs2/projects/sfcwinds/data/weather_data_expanded.db"
conn = sqlite3.connect(db_path)

print(f"Connected to database: {db_path}")

# Load the stations table into a DataFrame
try:
    stations_df = pd.read_sql_query("SELECT * FROM stations", conn)
    print("Successfully loaded 'stations' table.")
except Exception as e:
    print(f"Error loading 'stations' table: {e}")
    conn.close()
    exit()

stations_df.loc[stations_df.source_network == "NICEnet", "height"] = 3
stations_df.loc[stations_df.source_network == "NICEnet", "state"] = "NV"
stations_df.loc[stations_df['station_id'] == 'CBVA', 'state'] = 'CA'

#Station info data as dict keyed by station_id: (from https://nicenet.dri.edu/#aboutnicenet)
station_info = {
    "NPVA": {"station_name": "Paradise Valley", "begints": "Nov 2010"},
    "CBVA": {"station_name": "Bridgeport Valley", "begints": "Jul 2012"},
    "NCLV": {"station_name": "Clover Valley", "begints": "Sep 2011"},
    "NSTV": {"station_name": "Steptoe Valley North", "begints": "Mar 2011"},
    "NANV": {"station_name": "Antelope Valley", "begints": "Jun 2011"},
    "NSPA": {"station_name": "Truckee Meadows", "begints": "May 2010"},
    "NNSV": {"station_name": "North Spring Valley", "begints": "Jun 2011"},
    "NSWM": {"station_name": "Steptoe Valley WMA", "begints": "Mar 2011"},
    "NMAS": {"station_name": "Mason Valley WMA", "begints": "Apr 2010"},
    "NCVA": {"station_name": "Carson Valley", "begints": "Aug 2010"},
    "NSMV": {"station_name": "Smith Valley", "begints": "Aug 2010"},
    "NSNA": {"station_name": "Snake Valley", "begints": "Aug 2010"},
    "NSSV": {"station_name": "Sand Spring Valley", "begints": "Dec 2010"},
    "NPWL": {"station_name": "Pahranagat NWR", "begints": "Jul 2010"},
    "NMOA": {"station_name": "Moapa Valley", "begints": "Feb 2010"},
    "NROG": {"station_name": "Rogers Spring", "begints": "Sep 2010"},
    "NHUA": {"station_name": "Hualapai Flat", "begints": "Oct 2012"},
    "NREE": {"station_name": "Reese River Valley", "begints": "May 2014"},
}


# Map the 'station_name' using station_id
stations_df['station_name'] = stations_df['station_id'].map(lambda x: station_info.get(x, {}).get('station_name'))

# Map the 'begints' (install date) similarly
stations_df['begints'] = stations_df['station_id'].map(lambda x: station_info.get(x, {}).get('begints'))

# Convert 'begints' from string like 'Nov 2010' to datetime for consistency
stations_df['begints'] = pd.to_datetime(stations_df['begints'], format='%b %Y', errors='coerce')
stations_df['begints'] = stations_df['begints'].dt.strftime("%Y-%m-%d %H:%M:%S")

# --- Saving Changes Back to Database ---
print("\nSaving updated 'stations' table back to the database...")
try:
    stations_df.to_sql(name='stations', con=conn, if_exists='replace', index=False)
    print("### Database Update Complete")
except Exception as e:
    print(f"### Database Update Failed: {e}")

# Close the database connection
conn.close()


Connected to database: /kfs2/projects/sfcwinds/data/weather_data_expanded.db
Successfully loaded 'stations' table.

Saving updated 'stations' table back to the database...
### Database Update Complete


In [396]:
stations_df[stations_df.source_network == "NICEnet"]

,station_id,station_name,lat,lon,height,elev,begints,endts,source_network,state
127,NPVA,Paradise Valley,41.185556,-117.660278,3.0,1341.1200,2010-11-01 00:00:00,None,NICEnet,NV
128,NSMV,Smith Valley,38.751389,-119.335556,3.0,1488.9480,2010-08-01 00:00:00,None,NICEnet,NV
129,NCLV,Clover Valley,40.868333,-114.969444,3.0,1720.9008,2011-09-01 00:00:00,None,NICEnet,NV
130,NSSV,Sand Spring Valley,37.646667,-115.797500,3.0,1466.0880,2010-12-01 00:00:00,None,NICEnet,NV
131,NANV,Antelope Valley,40.093889,-117.168889,3.0,1484.6808,2011-06-01 00:00:00,None,NICEnet,NV
132,NNSV,North Spring Valley,39.458056,-114.506667,3.0,1719.6816,2011-06-01 00:00:00,None,NICEnet,NV
133,NSNA,Snake Valley,39.021111,-114.089722,3.0,1578.8640,2010-08-01 00:00:00,None,NICEnet,NV
134,NPWL,Pahranagat NWR,37.245556,-115.106389,3.0,982.9800,2010-07-01 00:00:00,None,NICEnet,NV
135,CBVA,Bridgeport Valley,38.223056,-119.252222,3.0,1979.6760,2012-07-01 00:00:00,None,NICEnet,CA
136,NSWM,Steptoe Valley WMA,39.195556,-114.810278,3.0,1965.9600,2011-03-01 00:00:00,None,NICEnet,NV


In [397]:
# Number of stations in NM
len(stations_df[stations_df.state == "NM"]) # , source_network == "*"]

152

In [398]:
# Number of stations in NM
stations_df[~stations_df['state'].isin(["AK", "HI", "AS", "GU", "PR", "VI"])] # without Alaska, Hawaii and overseas

,station_id,station_name,lat,lon,height,elev,begints,endts,source_network,state
0,EAS,None,37.871000,-97.205000,2.00,427.0,None,None,None,None
1,HES,None,38.133300,-97.400000,2.00,457.2,None,None,None,None
2,MOB,None,37.773344,-100.952989,2.00,894.1,None,None,None,None
3,POW,None,39.800000,-95.800000,2.00,365.8,None,None,None,None
4,SCO,None,38.534669,-100.730783,2.00,890.0,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
3280,US-NR3,Niwot Ridge Alpine (T-Van West),40.052000,-105.586400,3.00,3504.0,2020-07-05 00:00:00,None,AmeriFlux,CO
3281,US-ASH,USSL San Joaquin Valley Almond High Salinity,36.169700,-120.201000,8.01,147.0,None,None,AmeriFlux,CA
3283,US-Ro5,Rosemount I18_South,44.691000,-93.057600,3.25,283.0,2018-07-09 00:00:00,None,AmeriFlux,MN
3284,US-MN3,"Morris: Corn-Soybean, Conventional Tillage",45.609100,-96.126500,4.57,339.0,2020-06-29 00:00:00,None,AmeriFlux,MN


# Synoptic data

In [231]:
# Synoptic stations

synoptic_csv = "/kfs2/projects/sfcwinds/data/synoptic/US_stations_under_10m.csv"

df_synoptic = pd.read_csv(synoptic_csv)
df_synoptic['STID'] = df_synoptic['STID'].astype(str).str.upper()
synoptic_station_ids = set(df_synoptic['STID'])


In [269]:
network = "USCRN"   # no USCRN in synoptic

print(df_synoptic[df_synoptic.network == network])

len(df_synoptic[df_synoptic.network == network])


Empty DataFrame
Columns: [STID, name, state, network, network_long, Wind Sensor Height (m), lat, lon, Start Time, End Time, obs_time, obs_time_years]
Index: []


0

In [270]:
df_synoptic.network.unique()

array(['UUNET', 'BTAVAL', 'UTAH DOT', 'AGRIMET', 'FGNet', 'RAWS', 'HMMN',
       'AZMET', 'HADS', 'SCAN', 'DRI', 'CRN', 'NJNET',
       'UTAH CLIMATE CENTER', 'SDGE', 'DEOS', 'NMCC', 'MOCOMAGNET',
       'UCC-AGNET', 'USARRAY', 'MISC', 'AVALANCHE', 'CoAgMet', 'TWDB',
       'IUTAH', 'MT-MESO', 'UGA', 'CCU', 'MAWN', 'IEM', 'PEMN', 'IGS',
       'LSU-AGNET', 'CLNP', 'PURDUE'], dtype=object)

In [271]:
# Networks in the database, but not in synoptic
stations_df[stations_df.station_id.str.upper().isin(stations_in_df - set(df_synoptic.STID))].source_network.unique()

array([None, 'CoAgMet', 'NICEnet', 'AmeriFlux'], dtype=object)

In [272]:
network = "AZMET"   # This is the same stations as in our own AZMet - > own is left out.

df_synoptic[df_synoptic.network == network]


,STID,name,state,network,network_long,Wind Sensor Height (m),lat,lon,Start Time,End Time,obs_time,obs_time_years
709,AZM07,Aguila,AZ,AZMET,The Arizona Meteorological Network,3.0,33.94619,-113.18651,1997-01-01T00:00:00Z,2025-04-04T21:00:00Z,10320 days 21:00:00,28.257016
710,AZM09,Bonita,AZ,AZMET,The Arizona Meteorological Network,3.0,32.47748,-109.93273,1997-02-02T00:00:00Z,2025-04-04T21:00:00Z,10288 days 21:00:00,28.169405
711,AZM26,Buckeye,AZ,AZMET,The Arizona Meteorological Network,3.0,33.40919,-112.67784,1998-01-24T00:00:00Z,2025-04-04T21:00:00Z,9932 days 21:00:00,27.194730
712,AZM05,Coolidge,AZ,AZMET,The Arizona Meteorological Network,3.0,32.97999,-111.60568,1997-01-01T00:00:00Z,2025-04-04T21:00:00Z,10320 days 21:00:00,28.257016
713,AZM23,Harquahala,AZ,AZMET,The Arizona Meteorological Network,3.0,33.49073,-113.11125,1997-01-01T00:00:00Z,2025-04-04T21:00:00Z,10320 days 21:00:00,28.257016
714,AZM06,Maricopa,AZ,AZMET,The Arizona Meteorological Network,3.0,33.06894,-111.97224,1997-01-01T00:00:00Z,2025-04-04T21:00:00Z,10320 days 21:00:00,28.257016
715,AZM20,Mohave,AZ,AZMET,The Arizona Meteorological Network,3.0,34.96746,-114.61108,1997-01-01T00:00:00Z,2025-04-04T21:00:00Z,10320 days 21:00:00,28.257016
716,AZM19,Paloma,AZ,AZMET,The Arizona Meteorological Network,3.0,32.92715,-112.89674,1997-01-01T00:00:00Z,2025-04-04T21:00:00Z,10320 days 21:00:00,28.257016
717,AZM08,Parker,AZ,AZMET,The Arizona Meteorological Network,3.0,33.96430,-114.48550,1997-01-01T00:00:00Z,2025-04-04T21:00:00Z,10320 days 21:00:00,28.257016
718,AZM15,Phoenix Encanto,AZ,AZMET,The Arizona Meteorological Network,3.0,33.47844,-112.09805,1997-01-01T00:00:00Z,2025-04-04T21:00:00Z,10320 days 21:00:00,28.257016


# Observation files

In [447]:
# --- Count stations with Parquet obs ---
outputs_path = "/kfs2/projects/sfcwinds/outputs"

stations_with_obs = set()
for network in os.listdir(outputs_path):
    network_path = os.path.join(outputs_path, network)
    if not os.path.isdir(network_path):
        continue

    for station in os.listdir(network_path):
        station_path = os.path.join(network_path, station)
        
        if os.path.isdir(station_path):
            has_parquet = any(f.endswith(".parquet") for f in os.listdir(station_path))
            if has_parquet:
                stations_with_obs.add(station.upper())

obs_count = len(stations_with_obs)

print(f"Total stations with observation data:    {obs_count}")  # there are no duplicate staion names.

Total stations with observation data:    3278


# Compare metadata between sources

In [435]:
# Upper case
stations_df.station_id = stations_df.station_id.str.upper()  # obs have only upper case letters

In [436]:
# Drop duplicates
stations_df = stations_df.drop_duplicates(subset='station_id', keep='first')

In [448]:
stations_in_df = set(stations_df.station_id)

common_stations = stations_with_obs & stations_in_df
only_in_obs = stations_with_obs - stations_in_df
only_in_df = stations_in_df - stations_with_obs

print(f"Total in observations: {len(stations_with_obs)}")
print(f"Total in station metadata: {len(stations_in_df)}")
print(f"Stations in both: {len(common_stations)}")
print(f"Stations only in obs: {len(only_in_obs)}")
print(f"Stations only in metadata: {len(only_in_df)}")


Total in observations: 3278
Total in station metadata: 3262
Stations in both: 3262
Stations only in obs: 16
Stations only in metadata: 0


In [438]:
# Stations in the observations, but not in database
only_in_obs  # for the Alabama USCRN data we don't have all metadata, so just skip these.

{'AL_BREWTON_3_NNE',
 'AL_CLANTON_2_NE',
 'AL_COURTLAND_2_WSW',
 'AL_CULLMAN_3_ENE',
 'AL_GAINESVILLE_2_NE',
 'AL_GREENSBORO_2_WNW',
 'AL_HIGHLAND_HOME_2_S',
 'AL_MUSCLE_SHOALS_2_N',
 'AL_NORTHPORT_2_S',
 'AL_RUSSELLVILLE_4_SSE',
 'AL_SCOTTSBORO_2_NE',
 'AL_SELMA_6_SSE',
 'AL_TALLADEGA_10_NNE',
 'AL_THOMASVILLE_2_S',
 'AL_TROY_2_W',
 'AL_VALLEY_HEAD_1_SSW',
 'ON_EGBERT_1_W',
 'SA_TIKSI_4_SSE'}

In [440]:
# Stations in the metadata database, but not in observations
stations_df[((stations_df.station_id.isin(only_in_df)))]  # these have been deleted.

,station_id,station_name,lat,lon,height,elev,begints,endts,source_network,state
0,EAS,None,37.871000,-97.205000,2.0,427.0000,None,None,None,None
1,HES,None,38.133300,-97.400000,2.0,457.2000,None,None,None,None
2,MOB,None,37.773344,-100.952989,2.0,894.1000,None,None,None,None
3,POW,None,39.800000,-95.800000,2.0,365.8000,None,None,None,None
4,SCO,None,38.534669,-100.730783,2.0,890.0000,None,None,None,None
240,WI_NECEDAH_5_WNW,None,44.060000,-90.170000,1.5,284.3800,None,None,USCRN,WI
289,VA_STERLING_0_N,None,38.970000,-77.480000,1.5,87.4800,None,None,USCRN,VA
1849,LPSN6,None,43.470160,-74.413070,6.1,548.6400,None,2025-04-04 22:00:00,RAWS,NY
1975,MKLN7,None,35.469350,-79.380960,6.1,96.6216,None,2025-04-04 21:15:00,RAWS,NC


In [444]:
# Remove metatada without obs
stations_df = stations_df[~stations_df['station_id'].isin(only_in_df)]

In [450]:
# save dataframe with final metadata

# save with HI, AK and overseas
output_path = "/kfs2/projects/sfcwinds/outputs/metadata_US.csv"
stations_df.to_csv(output_path, index=False)

# save CONUS
output_path = "/kfs2/projects/sfcwinds/outputs/metadata_CONUS.csv"
stations_df[~stations_df['state'].isin(["AK", "HI", "AS", "GU", "PR", "VI"])].to_csv(output_path, index=False)

In [453]:
""" final numbers """
print (f"Number of stations in US: {len(stations_df)}")
print (f"Number of stations in CONUS: {len(stations_df[~stations_df['state'].isin(["AK", "HI", "AS", "GU", "PR", "VI"])])}")

Number of stations in US: 3262
Number of stations in CONUS: 3007
